# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 15 2022 3:32PM,244199,10,0085938237,ISDIN Corporation,Released
1,Aug 15 2022 3:31PM,244198,21,623782,"NBTY Global, Inc.",Released
2,Aug 15 2022 3:29PM,244197,16,TASA-280053,TASA USA Inc.,Released
3,Aug 15 2022 3:28PM,244196,21,624082,"NBTY Global, Inc.",Released
4,Aug 15 2022 3:27PM,244195,10,0085938204,ISDIN Corporation,Released
5,Aug 15 2022 3:27PM,244195,10,0085938218,ISDIN Corporation,Released
6,Aug 15 2022 3:27PM,244195,10,0085938219,ISDIN Corporation,Released
7,Aug 15 2022 3:27PM,244195,10,0085938221,ISDIN Corporation,Released
8,Aug 15 2022 3:27PM,244195,10,0085938222,ISDIN Corporation,Released
9,Aug 15 2022 3:27PM,244195,10,0085938223,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
30,244195,Released,15
31,244196,Released,1
32,244197,Released,1
33,244198,Released,1
34,244199,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
244195,NaN,15.0
244196,NaN,1.0
244197,NaN,1.0
244198,NaN,1.0
244199,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
244122,0.0,6.0
244135,4.0,6.0
244137,0.0,1.0
244144,0.0,96.0
244145,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
244122,0,6
244135,4,6
244137,0,1
244144,0,96
244145,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,244122,0,6
1,244135,4,6
2,244137,0,1
3,244144,0,96
4,244145,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,244122,,6
1,244135,4,6
2,244137,,1
3,244144,,96
4,244145,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 15 2022 3:32PM,244199,10,ISDIN Corporation
1,Aug 15 2022 3:31PM,244198,21,"NBTY Global, Inc."
2,Aug 15 2022 3:29PM,244197,16,TASA USA Inc.
3,Aug 15 2022 3:28PM,244196,21,"NBTY Global, Inc."
4,Aug 15 2022 3:27PM,244195,10,ISDIN Corporation
19,Aug 15 2022 3:24PM,244194,21,"NBTY Global, Inc."
20,Aug 15 2022 3:22PM,244193,10,"Citieffe, Inc."
21,Aug 15 2022 3:20PM,244192,10,ISDIN Corporation
33,Aug 15 2022 3:18PM,244191,21,"NBTY Global, Inc."
34,Aug 15 2022 3:16PM,244190,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Aug 15 2022 3:32PM,244199,10,ISDIN Corporation,,1
1,Aug 15 2022 3:31PM,244198,21,"NBTY Global, Inc.",,1
2,Aug 15 2022 3:29PM,244197,16,TASA USA Inc.,,1
3,Aug 15 2022 3:28PM,244196,21,"NBTY Global, Inc.",,1
4,Aug 15 2022 3:27PM,244195,10,ISDIN Corporation,,15
5,Aug 15 2022 3:24PM,244194,21,"NBTY Global, Inc.",,1
6,Aug 15 2022 3:22PM,244193,10,"Citieffe, Inc.",,1
7,Aug 15 2022 3:20PM,244192,10,ISDIN Corporation,,12
8,Aug 15 2022 3:18PM,244191,21,"NBTY Global, Inc.",,1
9,Aug 15 2022 3:16PM,244190,21,"NBTY Global, Inc.",1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 15 2022 3:32PM,244199,10,ISDIN Corporation,1,
1,Aug 15 2022 3:31PM,244198,21,"NBTY Global, Inc.",1,
2,Aug 15 2022 3:29PM,244197,16,TASA USA Inc.,1,
3,Aug 15 2022 3:28PM,244196,21,"NBTY Global, Inc.",1,
4,Aug 15 2022 3:27PM,244195,10,ISDIN Corporation,15,
5,Aug 15 2022 3:24PM,244194,21,"NBTY Global, Inc.",1,
6,Aug 15 2022 3:22PM,244193,10,"Citieffe, Inc.",1,
7,Aug 15 2022 3:20PM,244192,10,ISDIN Corporation,12,
8,Aug 15 2022 3:18PM,244191,21,"NBTY Global, Inc.",1,
9,Aug 15 2022 3:16PM,244190,21,"NBTY Global, Inc.",,1


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 15 2022 3:32PM,244199,10,ISDIN Corporation,1,
1,Aug 15 2022 3:31PM,244198,21,"NBTY Global, Inc.",1,
2,Aug 15 2022 3:29PM,244197,16,TASA USA Inc.,1,
3,Aug 15 2022 3:28PM,244196,21,"NBTY Global, Inc.",1,
4,Aug 15 2022 3:27PM,244195,10,ISDIN Corporation,15,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Aug 15 2022 3:32PM,244199,10,ISDIN Corporation,1.0,NaN
1,Aug 15 2022 3:31PM,244198,21,"NBTY Global, Inc.",1.0,NaN
2,Aug 15 2022 3:29PM,244197,16,TASA USA Inc.,1.0,NaN
3,Aug 15 2022 3:28PM,244196,21,"NBTY Global, Inc.",1.0,NaN
4,Aug 15 2022 3:27PM,244195,10,ISDIN Corporation,15.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 15 2022 3:32PM,244199,10,ISDIN Corporation,1.0,0.0
1,Aug 15 2022 3:31PM,244198,21,"NBTY Global, Inc.",1.0,0.0
2,Aug 15 2022 3:29PM,244197,16,TASA USA Inc.,1.0,0.0
3,Aug 15 2022 3:28PM,244196,21,"NBTY Global, Inc.",1.0,0.0
4,Aug 15 2022 3:27PM,244195,10,ISDIN Corporation,15.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3906857,227.0,4.0
12,244179,0.0,1.0
16,732523,3.0,0.0
19,732501,1.0,8.0
20,732452,52.0,0.0
21,1220969,4.0,1.0
22,732452,3.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3906857,227.0,4.0
1,12,244179,0.0,1.0
2,16,732523,3.0,0.0
3,19,732501,1.0,8.0
4,20,732452,52.0,0.0
5,21,1220969,4.0,1.0
6,22,732452,3.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,227.0,4.0
1,12,0.0,1.0
2,16,3.0,0.0
3,19,1.0,8.0
4,20,52.0,0.0
5,21,4.0,1.0
6,22,3.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,227.0
1,12,Released,0.0
2,16,Released,3.0
3,19,Released,1.0
4,20,Released,52.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,21,22
Status,,,,,,,
Executing,4.0,1.0,0.0,8.0,0.0,1.0,0.0
Released,227.0,0.0,3.0,1.0,52.0,4.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,21,22
0,Executing,4.0,1.0,0.0,8.0,0.0,1.0,0.0
1,Released,227.0,0.0,3.0,1.0,52.0,4.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,21,22
0,Executing,4.0,1.0,0.0,8.0,0.0,1.0,0.0
1,Released,227.0,0.0,3.0,1.0,52.0,4.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()